# 🚀 SpaceX Launch Data Analysis using SQL
## This project demonstrates SQL querying skills on SpaceX mission data stored in a local SQLite database. We explore launch sites, payloads, mission outcomes, and landing results using complex SQL queries integrated within a Python workflow.

### 🔧 Step 1: Load the SQL Extension and Set Up the Environment 

In [1]:
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'  #set style

import csv, sqlite3
import pandas as pd

# Connect to SQLite database
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

# Bind the database connection to the SQL magic
%sql sqlite:///my_data1.db


### 📥 Step 2: Load and Clean SpaceX Data

In [2]:
# Load dataset from the cloud
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")

# Clean up whitespace in outcome field
df['Mission_Outcome'] = df['Mission_Outcome'].str.strip()

# Save to database
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")


101

#### Check landing outcomes:

In [3]:
df['Landing_Outcome'].value_counts()


Landing_Outcome
Success                   38
No attempt                21
Success (drone ship)      14
Success (ground pad)       9
Controlled (ocean)         5
Failure (drone ship)       5
Failure                    3
Uncontrolled (ocean)       2
Failure (parachute)        2
Precluded (drone ship)     1
No attempt                 1
Name: count, dtype: int64

### 🧹 Step 3: Create Clean Working Table
#### Drop the table if it already exists, and create a filtered version with non-null dates:

In [4]:
%sql DROP TABLE IF EXISTS SPACEXTABLE;

%sql CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL;


 * sqlite:///my_data1.db
Done.
 * sqlite:///my_data1.db
Done.


[]

### 🔍 SQL Analysis Tasks
#### ✅ Task 1: List all unique launch sites

In [5]:
%sql SELECT DISTINCT "Launch_Site" FROM SPACEXTBL;


 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


### ✅ Task 2: Show the first 5 launches from sites starting with "CCA"

In [6]:
%sql SELECT * FROM SPACEXTBL WHERE "Launch_Site" LIKE 'CCA%' LIMIT 5;


 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### ✅ Task 3: Total payload mass for NASA (CRS)

In [7]:
%sql SELECT SUM("PAYLOAD_MASS__KG_") FROM SPACEXTBL WHERE "Customer" = 'NASA (CRS)';


 * sqlite:///my_data1.db
Done.


SUM(PAYLOAD_MASS__KG_)
45596


### ✅ Task 4: Average payload mass for Booster Version 'F9 v1.1'

In [8]:
%sql SELECT AVG("PAYLOAD_MASS__KG_") FROM SPACEXTBL WHERE "Booster_Version" = 'F9 v1.1';


 * sqlite:///my_data1.db
Done.


AVG(PAYLOAD_MASS__KG_)
2928.4


### ✅ Task 5: Date of first successful ground pad landing

In [9]:
%sql SELECT MIN("Date") FROM SPACEXTBL WHERE "Landing_Outcome" = 'Success (ground pad)';


 * sqlite:///my_data1.db
Done.


MIN(Date)
2015-12-22


### ✅ Task 6: Boosters that had successful drone ship landings with payload between 4000–6000 kg

In [10]:
%%sql SELECT DISTINCT "Booster_Version" 
FROM SPACEXTBL 
WHERE "Landing_Outcome" = 'Success (drone ship)' 
AND "PAYLOAD_MASS__KG_" BETWEEN 4000 AND 6000;


 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### ✅ Task 7: Count of missions by mission outcome

In [11]:
%%sql SELECT "Mission_Outcome", COUNT(*) as Total 
     FROM SPACEXTBL 
     GROUP BY "Mission_Outcome";


 * sqlite:///my_data1.db
Done.


Mission_Outcome,Total
Failure (in flight),1
Success,99
Success (payload status unclear),1


### ✅ Task 8: Boosters with the highest payload mass ever carried

In [12]:
%%sql SELECT DISTINCT "Booster_Version" 
     FROM SPACEXTBL 
     WHERE "PAYLOAD_MASS__KG_" = (
         SELECT MAX("PAYLOAD_MASS__KG_") FROM SPACEXTBL
     );


 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### ✅ Task 9: Landing outcome by month for 2015 (with month name)

In [13]:
%%sql SELECT 
    CASE 
        WHEN substr("Date", 6, 2) = '01' THEN 'January'
        WHEN substr("Date", 6, 2) = '02' THEN 'February'
        WHEN substr("Date", 6, 2) = '03' THEN 'March'
        WHEN substr("Date", 6, 2) = '04' THEN 'April'
        WHEN substr("Date", 6, 2) = '05' THEN 'May'
        WHEN substr("Date", 6, 2) = '06' THEN 'June'
        WHEN substr("Date", 6, 2) = '07' THEN 'July'
        WHEN substr("Date", 6, 2) = '08' THEN 'August'
        WHEN substr("Date", 6, 2) = '09' THEN 'September'
        WHEN substr("Date", 6, 2) = '10' THEN 'October'
        WHEN substr("Date", 6, 2) = '11' THEN 'November'
        WHEN substr("Date", 6, 2) = '12' THEN 'December'
    END AS "Month_Name", 
    "Landing_Outcome", 
    "Booster_Version", 
    "Launch_Site" 
FROM SPACEXTBL 
WHERE substr("Date", 0, 5) = '2015' 
AND "Landing_Outcome" = 'Failure (drone ship)';


 * sqlite:///my_data1.db
Done.


Month_Name,Landing_Outcome,Booster_Version,Launch_Site
January,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### ✅ Task 10: Count of landing outcomes between 2010-06-04 and 2017-03-20

In [14]:
%%sql SELECT "Landing_Outcome", COUNT(*) as "Count" 
     FROM SPACEXTBL 
     WHERE "Date" BETWEEN '2010-06-04' AND '2017-03-20' 
     GROUP BY "Landing_Outcome" 
     ORDER BY "Count" DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


## ✅ Conclusion
## This analysis demonstrates how SQL can be used within a Python environment to perform real-world queries on structured data. From cleaning and preparing data, to extracting business-relevant insights, this project showcases end-to-end SQL data analysis.

